# Mapping barcodes and promoters

In [22]:
using wgregseq, CSV, DataFrames, Glob, BioSequences, CairoMakie
import wgregseq

wgregseq.plotting_style.default_makie!()

In [10]:
df = CSV.read(
    "../../../data/barcodes/20220514_mapping/110_collapsed.txt", 
    DataFrame, 
    delim=" ", 
    ignorerepeated=true, 
    header=[:counts, :barcode, :promoter]
)

df.barcode = map(x -> LongDNA{4}(x), df.barcode)
df.promoter = map(x -> LongDNA{4}(x), df.promoter)

first(df, 5)

,counts,barcode,promoter
,Int64,LongSequ…,LongSequ…
1,12585,TTATTAGGATCCCTCACCAA,CGGGGATGTTTTGGCGCGTTACTTGCGCTTCTAGTTTGGTTTTTCGTGCCGTATGTTCGTGCGGTTACAGGCGTTAGATTCACCTACTTTTGTGAATGTATGTAGCATAGCTCAACGACAATATAAACGCAGTAATGGGTTTATTAACTTTTGACCATTG
2,11223,GTCCCGATGGGCACCGATAA,TTTGACTGCCGATTGAGAAGTCATGTGTTAAATGGCGGCACATACATGCACTATGGCACATAACGCCAACCTCTTGCCGTAGCGGCTTCTGCTAGTATCCGCAATAATTTTACATTGTAATCGCGCTAAATACTGCTTCACCACAAGGAATTCAAATGAA
3,8181,GGCGGACTCTTTGACTTTCG,GCCTGCATGTCAAAGTCCTGGTAAAAGTCTTATTGTGACGGAAAACGAACGCCACGCAAAGCTAACCGGACAATAGTGTAGTGCTTTTCTGTGCTTAGCGGTTAGAATATTCTCATCACTATATCTGGCGTTGTCCATGTTATAGTTATTAAAAAGTTTA
4,7576,CAATGCTGTCGAACACATTC,CAGCCCATAGAGGAACTGCAACTATTGCAAATGAAGAAATTAAGCTGGATGGCATCGCGCAATGAGGAAGATTGGCCGGGGCTGACAGTAAATAGCAATCGTAGAATCAGCCGCAAGCCCGAAAGATTCCAGTGAGGCGCAATGTTCCGGGGGCTTTAGC
5,6880,ATTCGGTATTTAATATCGCC,TGGATCTGACTGGAGTGAGATAGGCATCATCTAACGCATCGTCGCTAATACCATACTAAAACGATTGTAACGACGTGCTGCTTTCATAACCTGCACGTACTGTATGATTATCCAGTTAGCTCTGAGGCGTTTTCACGCTGGCAAGGCGCATAAACTCTTT


In [31]:
df_seqs = CSV.read(
    "../../../data/twist_orders/2022-02-15_twist_order.csv", 
    DataFrame,
    types=Dict(
        "fwd_primer" => String,
        "rev_primer1" => String,
        "rev_primer2" => String,
        "rev_primer3" => String,
    )
)

df_seqs.fwd_primer = parse.(Tuple{Int64, Tuple{Int64, Int64}}, df_seqs.fwd_primer)
df_seqs.rev_primer1 = parse.(Tuple{Int64, Tuple{Int64, Int64}}, df_seqs.rev_primer1)
df_seqs.rev_primer2 = parse.(Tuple{Int64, Tuple{Int64, Int64}}, df_seqs.rev_primer2)
df_seqs.rev_primer3 = parse.(Tuple{Int64, Tuple{Int64, Int64}}, df_seqs.rev_primer3)


df_seqs.genes = parse.(Vector{String}, df_seqs.genes)
df_seqs.sequence = [LongDNA{4}(seq) for seq in df_seqs.sequence]

first(df_seqs, 5)

,genes,sequence
,Array…,LongSequ…
1,"[""rspA"", ""rspB""]",ACCTGTAATTCCAAGCGTCTACTAGTTTTCATCTTTTGTCAACCATTCACAGCGCAAATATACGCCTTTTTTTGTGATCACTCCGGCTTTTTTCGATCTTTATACTTGTATGGTAGTAGCTCAGTTGCGTAGATTTCATGCATCACGACAAGCGATGCAAGGAATCGAACATGAAGATCGTAAAGGGGGCCCGGGCACAGCAATCAAAAGTCGTGTAAAATCCGAGAACCGAATACGTGCCTGAGACAAC
2,"[""rspA"", ""rspB""]",ACCTGTAATTCCAAGCGTCTACTAGTTCTGATCTTTTGTAAATCATTCACAGTGCAAATTTAAGTCTTTTTTTGTGATCACTCCGGCTATATTCGATCTATTTACTTGTATGGTAGTAGCTCAGTTGCGTAGATTTCATGCGTCACGACAAGCGACGCAGGGAATCGAATATGAAGATCGTAAAGGGGGCCCGGGCACAGCAATCAAAAGTCGTGTAAAATCCGAGAACCGAATACGTGCCTGAGACAAC
3,"[""rspA"", ""rspB""]",ACCTGTAATTCCAAGCGTCTACTAGTTTTCATCTTTTCTCAACCACTCACAGCGTAAACTTACGCCTGTTTGTGTGATCACTACGGCGTTTTTCAATCTTTATACTTGTATGGTAGTCTCTCAGTTGCGTAGATTTGCTGCATCACGACAAGCGATGCATGGAATCGAACATGAAGGTCGTAAAGGGGGCCCGGGCACAGCAATCAAAAGTCGTGTAAAATCCGAGAACCGAATACGTGCCTGAGACAAC
4,"[""rspA"", ""rspB""]",ACCTGTAATTCCAAGCGTCTACTAGTGTTCATCCTTCTTCAACCATTCACAGCGGAAATATACGCCTTTTCTCGAGATCACTCCGGCTTTTTTCGATCTTTATACTTGTATGGTAGTAGCTGAGTTGTGTAGTTTTCATGCATCACAACACGCGATGCAAGGAATGAAACATGAGGATCGTAAAGGGGGCCCGGGCACAGCAATCAAAAGTCGTGTAAAATCCGAGAACCGAATACGTGCCTGAGACAAC
5,"[""rspA"", ""rspB""]",ACCTGTAATTCCAAGCGTCTACTAGTTTTCATCTTTTGTCAACCATTCACCGCGCAAATATACGCCTTCTTTTGTGATCACTCAGGCTCTTTTCGATCTTTCTACTAGTATGGTAATAGCTAATTTGCGTGGATTGCATGCATCACGACAAGCATTGCAAGGAATCGAACACGAAGATCCTTAAGGGGGCCCGGGCACAGCAATCAAAAGTCGTGTAAAATCCGAGAACCGAATACGTGCCTGAGACAAC


In [38]:
df_seqs_110 = df_seqs[map(x -> x[1] .== 110, df_seqs.rev_primer2), :]
dict_110 = Dict(df_seqs_110.sequence .=>df_seqs_110.promoter);

In [ ]:
name_list = String[]

for i in 1:nrow(df)
    try
        push!(name_list, sequence_dict[df.sequence[i]])
    catch e
        push!(name_list, "None")
    end
end

name_list |> unique